In [4]:
!pip install selenium
!pip install bs4
!pip install cx_Oracle

In [1]:
from selenium import webdriver # 셀리늄 라이브러리
from bs4 import BeautifulSoup
import time
import cx_Oracle # Oracle DB

# 오라클 접속
conn = cx_Oracle.connect('admin/qlxmaos12@database-2.cwmwyrme2oij.ap-northeast-2.rds.amazonaws.com/ORCL')

# 크롬 드라이버 객체 생성
driver = webdriver.Chrome('C:/Users/bitcamp/chromedriver.exe')
# 개발자 모드로 들어갈 URL
driver.get('https://hotel.naver.com/hotels/list?destination=place:Hong_Kong&rooms=2')
time.sleep(2) # 페이지가 로드 될 때까지 기다려야함

# 첫번째 반복문 - 페이지 수
for temp in range(1, 5):
    # 두번째 반복문 - 페이지 안에 호텔 리스트 수
    for i in range(1, 20):     
        # hotel_content 테이블에 저장할 리스트 생성
        kor_name = []
        eng_name = []
        main_img_url = []
        price = []
        content = []
        address = []
        star = []
        
        # hotel_images 테이블에 저장할 리스트 생성
        imgs = []
        
        # hotel_rooms 테이블에 저장할 리스트 생성
        room_img = []
        room_title = []
        room_qty = []
        
        # 텍스트로 변환 시킨 후 들어갈 리스트
        kor_title_total = []
        eng_title_total = []
        main_img_total = []
        price_total = []
        content_total = []
        address_total = []
        star_total = []
        
        imgs_total = []
        
        room_img_total = []
        room_title_total = []
        room_qty_total = []
        
        content_items = []
        img_items = []
        rooms_items = []
        
        # 호텔 리스트 가져오기
        hotel_list = driver.find_elements_by_css_selector('.lnk_hotel')
        # 한개씩 돌면서 클릭
        hotel_list[i].click()
        time.sleep(1) # 로드 될때까지 기다림

        # 호텔리스트를 클릭하면 새탭으로 열려서 작업할 탭 타겟 변경
        driver.switch_to_window(driver.window_handles[1])
        # content 쪽에 더보기 버튼이 있으면 클릭 없으면 그냥 수집
        if driver.find_elements_by_xpath("//a[@class = 'more ng-scope']"):
            driver.find_elements_by_xpath("//a[@class = 'more ng-scope']")[0].click()

        # 호텔 한글이름 텍스트로 수집
        kor_name = driver.find_elements_by_xpath("//strong[@class = 'hotel_name_ko ng-binding ng-scope']")[0].text
        # 호텔 영어이름 수집
        eng_name = driver.find_elements_by_xpath("//strong[@class = 'hotel_name_en ng-binding']")
        # 메인사진 url 수집
        main_img_url = driver.find_elements_by_xpath("//div[@class = 'flick-content thumb0']")[0].find_elements_by_css_selector('img')[0].get_attribute('src')
        # 가격 수집
        price = driver.find_elements_by_xpath("//strong[@class = 'price ng-scope']")[0].find_elements_by_css_selector('.ng-binding')[0].text
        # 내용 수집 (수집안되는 문자를 위해 'ignore'를 써서 무시함)
        content = driver.find_elements_by_xpath("//p[@class = 'txt ng-binding']")[0].text.encode('euc-kr', 'ignore').decode('euc-kr')
        # 주소 수집
        address = driver.find_elements_by_xpath("//p[@class = 'addr ng-binding ng-scope']")
        # 호텔 등급 수집
        star = driver.find_elements_by_xpath("//span[@class = 'grade ng-binding ng-scope']")
        
        kor_title_total.append(kor_name)
        main_img_total.append(main_img_url)
        price_total.append(price)
        content_total.append(content)
        
        # 영어이름이 들어가는 HTML 태그가 있으면
        if eng_name:
            eng_title_total.append(eng_name[0].text)
        # 없으면
        
        elif not eng_name:
            eng_title_total.append('null')

        if address:
            address_total.append(address[0].text)
        elif not address:
            address_total.append('null')

        if star:
            star_total.append(star[0].text)
        elif not star:
            star_total.append('null')

        # 이미지가 퍼져있는 형태면 실행
        if driver.find_elements_by_xpath("//a[@class = 'btn_thumb_item']"):
            imgs = driver.find_elements_by_xpath("//a[@class = 'btn_thumb_item']")
            for i in range(len(imgs)):
                imgs_total.append(imgs[i].find_elements_by_css_selector('img')[0].get_attribute('src'))
        elif not driver.find_elements_by_xpath("//a[@class = 'btn_thumb_item']"):
            imgs = driver.find_elements_by_xpath("//div[@class = 'lst_flick']")[0].find_elements_by_css_selector('img')
            for i in range(len(imgs)):
                imgs_total.append(imgs[i].get_attribute('src'))
            
            #imgs_total.extend([i.text for i in imgs])
        
        # 방 이미지 url
        room_img = driver.find_elements_by_xpath("//a[@class = 'photo ng-scope']")            
        for i in range(len(room_img)):
            room_img_total.append(room_img[i].find_elements_by_css_selector('.img')[0].get_attribute('style')[23 : -3])
            
        # 방 이름
        room_title = driver.find_elements_by_xpath("//i[@class = 'room_name ng-binding']")        
        for i in range(len(room_title)):
            room_title_total.append(room_title[i].text)
            
        # 방 갯수
        room_qty = driver.find_elements_by_xpath("//i[@class = 'room_count ng-binding']")
        for i in range(len(room_qty)):
            room_qty_total.append(room_qty[i].text[1:-1])
            
        # 오라클 데이터베이스 커서 생성
        cursor = conn.cursor()
        content_sql = 'insert into hotel_content values(hnb_seq.nextval, :1, :2, :3, :4, :5, :6, :7)'
        
        for i in range(len(kor_title_total)):
            # 리스트 안에 튜플형태로 넣어줌
            content_items.append((kor_title_total[i], eng_title_total[i], star_total[i], address_total[i], price_total[i], content_total[i], main_img_total[i]))
        cursor.arraysize = len(content_items)
        # 첫번째 인자 - sql문, 두번째 인자 - 데이터
        cursor.executemany(content_sql, content_items)
        
        conn.commit()
        
        content_select_sql = "select hnb from hotel_content where kor_name = '" + kor_title_total[0] + "'"
        cursor.execute(content_select_sql)
        
        for row in cursor:
            hnb = row[0]
        
        imgs_sql = 'insert into hotel_images values(inb_seq.nextval, :1, :2)'
        
        for i in range(len(imgs_total)):
            img_items.append((hnb, imgs_total[i]))
        cursor.arraysize = len(img_items)
        cursor.executemany(imgs_sql, img_items)
            
        conn.commit()
        
        rooms_sql = 'insert into hotel_rooms values(rnb_seq.nextval, :1, :2, :3, :4)'
        
        for i in range(len(room_img_total)):
            rooms_items.append((hnb, room_title_total[i], room_qty_total[i], room_img_total[i]))
        cursor.arraysize = len(rooms_items)
        cursor.executemany(rooms_sql, rooms_items)
        
        conn.commit()
        
        cursor.close()
        driver.close()
        # 해당 탭의 정보 다 긁고 난 후 원래 탭쪽으로 포커스 이동
        driver.switch_to_window(driver.window_handles[0])
        
    # 다음 페이지 클릭
    driver.find_elements_by_css_selector("span.ng-scope > a.ng-binding")[temp].click()
    time.sleep(1)
    
conn.close()

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: DeprecationWarning: use driver.switch_to.window instead
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:141: DeprecationWarning: use driver.switch_to.window instead
